# VacationPy

In [1]:
# Dependencies and Setup
from IPython.display import display
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import 'API' and 'G' Keys
from config import gkey
from config import (api_key)

### Store Part I results into DataFrame
* Load the CSV that was exported in Part-I, to a DataFrame

In [2]:
# Dependencies and Setup
pro_dir = ''

cities_path = "../selected_cities/selected_cities.csv"

# Read the 'Cities' data that we just stored in a CSV file
cities_data = pd.read_csv(pro_dir + cities_path)

locations = cities_data[['Latitude', 'Longitude']]
hum_weights = cities_data['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights = hum_weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
cities_utemp_limit = cities_data[cities_data['Max Temp'] < 27]
cities_ultemp_limit = cities_utemp_limit[cities_utemp_limit['Max Temp'] > 21]
cities_ftemp_fws = cities_ultemp_limit[cities_ultemp_limit['Wind Speed'] < 10]
cities_filtered = cities_ftemp_fws[cities_ftemp_fws['Cloudiness'] == 0]
display(cities_filtered)
targ_rad = 5000

,City Name,Latitude,Longitude,Temperature,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,san rafael,-34.6177,-68.3301,23.03,23.03,23.03,38,0,4.15,AR,1610284201
83,hurghada,27.2574,33.8129,26.00,26.00,26.00,36,0,8.75,EG,1610284027
184,gasa,31.5000,34.4667,21.53,22.78,20.00,49,0,3.28,PS,1610284388
239,paita,-5.0892,-81.1144,22.00,22.00,22.00,83,0,6.17,PE,1610284456
317,kutum,14.2000,24.6667,25.81,25.81,25.81,15,0,6.78,SD,1610284563
321,saldanha,-33.0117,17.9442,26.00,26.00,26.00,36,0,9.77,ZA,1610284570
331,salalah,17.0151,54.0924,24.00,24.00,24.00,38,0,4.12,OM,1610284583
340,parkes,-33.1333,148.1833,23.89,23.89,23.89,36,0,0.45,AU,1610284597
395,marsa matruh,31.3525,27.2453,24.00,24.00,24.00,33,0,4.12,EG,1610284672
407,mandurah,-32.5500,115.7000,21.67,21.67,21.67,71,0,3.58,AU,1610284605


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
final_rest_data = {'Lat':[],
                   'Lng':[],
                   'Hotel Name':[],
                   'City':[],
                   'Country':[]
                  }

for index, row in cities_filtered.iterrows():
    coordinates = str(row['Latitude']) + ', ' + str(row['Longitude'])
    params = {"location": coordinates,
              "type": "restaurant",
              "keyword": "Chinese",
              'radius': targ_rad,
              "key": gkey
             }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params)
    restaurants_places = response.json()
    
    try:
        rest_data = restaurants_places['results'][0]
        rs_nm = rest_data['name']
        final_rest_data['Hotel Name'].append(rest_data['name'])
        final_rest_data['Lat'].append(restaurants_places['results'][0]['geometry']['location']['lat'])
        final_rest_data['Lng'].append(restaurants_places['results'][0]['geometry']['location']['lng'])
        final_rest_data['City'].append(row['City Name'])
        final_rest_data['Country'].append(row['Country'])
    except:
        aaa = 0
        hotel_df = pd.DataFrame(final_rest_data)

display(hotel_df)

,Lat,Lng,Hotel Name,City,Country
0,-34.609262,-68.349716,Chunghwa RESTO,san rafael,AR
1,27.218297,33.838522,China Town Restaurant,hurghada,EG
2,31.523542,34.444459,مطعم الشاورما التركي,gasa,PS
3,-5.088830,-81.095947,Chifa Restaurant Tay Wan,paita,PE
4,-33.003745,17.944751,Ni Hao Chinese Restaurant,saldanha,ZA
5,17.015697,54.094373,Chinese Cascade Chinese Restaurant 23289844,salalah,OM
6,-33.134281,148.174000,Fu Yuan Chinese Restaurant,parkes,AU
7,31.353140,27.240353,Magdy Seafood,marsa matruh,EG
8,-32.525975,115.722556,Mei Jing Chinese Restaurant,mandurah,AU


In [5]:
# NOTE: Do not change any of the code in this cell

# print(len(rest_locations),len(rest_names))

# Using the template, add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: Be sure to update with your DataFrame name

#hotel_infoa = ['\033[1m'+'Name\n'+row['Hotel Name']+'\n\033[1m'+'City\n'+row['City']+'\n\033[1m'+'Country\n'+row['Country'] for index,row in hotel_df.iterrows()]
hotel_info_b = ['Name \n' + row['Hotel Name'] + '\n City \n' + row['City'] + '\n Country \n' + row['Country'] for index, row in hotel_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, hover_text = hotel_info_b)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))